In [ ]:
from PIL import Image
import numpy as np

from os import listdir
from os.path import isfile, join



In [ ]:
IMAGE_DIR = "Images"
image_files = [f"{IMAGE_DIR}/{f}" for f in listdir(IMAGE_DIR) if isfile(join(IMAGE_DIR, f))]
image_files


In [ ]:
image_path = image_files[1]
Im = Image.open(image_path) #Image Loaded
I = np.asarray(Im) #Matrix representation of array
Im

In [ ]:
# f ( x, μ, σ) = 1 / (σ * sqrt(2 * pi)) * e^(- (x - μ)^2 / (2 * σ^2))

def normal(x, mu, sigma):
    exponent = -(x - mu)**2 / (2 * sigma**2)
    return  1 / (np.sqrt(2 * np.pi) * sigma) * np.exp(exponent)

def centered_array(size):
    if size % 2 == 0:
        raise ValueError("Size should be an odd integer.")
    half_size = size // 2
    centered_array = np.arange(-half_size, half_size + 1)
    return centered_array

def gaussian_mask(size, sigma):
    x = centered_array(size)
    mask = normal(x,0,sigma)
    mask /= np.sum(mask) #Normalize Values
    return mask
    
def convolution(image, kernel, dir="x", mode="zero"):
    if len(kernel.shape) == 1:
        kernel = np.array([kernel]) #Insert Empty Dim For X convulation
    if dir == "y":
        kernel = np.transpose(kernel)
        
    result = np.zeros_like(image)
    kernel_height, kernel_width = kernel.shape
    image_height, image_width = image.shape


    # Calculate padding size based on the kernel size
    padding_height = kernel_height // 2
    padding_width = kernel_width // 2
    padded_image = None

    if mode == "zero":
        padded_image = np.pad(image, ((padding_height, padding_height), (padding_width, padding_width)), mode='constant')
    elif mode == "reflect":
        padded_image = np.pad(image, ((padding_height, padding_height), (padding_width, padding_width)), mode='reflect')


    for y in range(image_height):
        for x in range(image_width):
            image_patch = padded_image[y:y+kernel_height, x:x+kernel_width]
            result[y, x] = np.sum(image_patch * kernel)
    return result

    
def elementwise_magnitude(m1, m2):
    result = np.sqrt((m1 ** 2) + (m2 ** 2))
    return result
def grad_direction(m1,m2):
    return np.arctan2(m1, m2)
    
# def non_maximum_suppression(magnitude, gradient_direction):
#     # Define the directions for 8 neighboring pixels
#     DIRECTIONS = [0, 45, 90, 135]
    
#     # Initialize the result with zeros
#     suppressed = np.zeros_like(magnitude, dtype=np.uint8)
    
#     # Iterate through each pixel in the magnitude image
#     rows, cols = magnitude.shape
#     for i in range(1, rows - 1):
#         for j in range(1, cols - 1):
#             angle = gradient_direction[i, j]
            
#             # Check the neighboring pixels in the gradient direction
#             if angle == 0 and magnitude[i, j] >= magnitude[i, j + 1] and magnitude[i, j] >= magnitude[i, j - 1]:
#                 suppressed[i, j] = magnitude[i, j]
#             elif angle == 45 and magnitude[i, j] >= magnitude[i - 1, j + 1] and magnitude[i, j] >= magnitude[i + 1, j - 1]:
#                 suppressed[i, j] = magnitude[i, j]
#             elif angle == 90 and magnitude[i, j] >= magnitude[i - 1, j] and magnitude[i, j] >= magnitude[i + 1, j]:
#                 suppressed[i, j] = magnitude[i, j]
#             elif angle == 135 and magnitude[i, j] >= magnitude[i - 1, j - 1] and magnitude[i, j] >= magnitude[i + 1, j + 1]:
#                 suppressed[i, j] = magnitude[i, j]
    
#     return suppressed


In [ ]:
from scipy.signal import convolve

mag = 1
kernel_size = 3
sigma = 5

G = gaussian_mask(kernel_size, sigma) # Gaussian X Filter
dx_mask = np.array([-mag, 0, mag]) #Central derivative Mask; Todo:: Determine what mask to use here

Gx = convolve(G, dx_mask) #todo:: can we use this convolve function
Gy =  np.transpose(np.array([Gx]))


In [ ]:
import cv2

result = cv2.filter2D(I, -1, Gx)
Test1 = Image.fromarray(result, mode='L')
Test1

result = cv2.filter2D(I, -1, Gy)
Test2 = Image.fromarray(result, mode='L')
Test2
Display(Test1, Test2)

In [ ]:
Ix = convolution(I, G)
Iy = convolution(I, G, dir='y')

I_dx = convolution(I, Gx)
I_dx = np.uint8(np.absolute(I_dx))
I_dy = convolution(I, Gy)
I_dy = np.uint8(np.absolute(I_dy))

I_mag = elementwise_magnitude(I_dx, I_dy)
I_mag = ((I_mag - I_mag.min()) / (I_mag.max() - I_mag.min()) * 255).astype(np.uint8)

I_deg = np.degrees(grad_direction(I_dx, I_dy))
I_non = non_maximum_suppression(I_mag, I_deg)
I_non = ((I_non - I_non.min()) / (I_non.max() - I_non.min()) * 255).astype(np.uint8)

Image_Gaus_x = Image.fromarray(Ix, mode='L')
Image_Gaus_y = Image.fromarray(Iy, mode='L') #Tranpose to apply in Y dir

Image_Gaus_dx = Image.fromarray(I_dx, mode='L') 
Image_Gaus_dy = Image.fromarray(I_dy, mode='L') 
Image_magnitude = Image.fromarray(I_mag, mode='L') 

Image_supressed = Image.fromarray(I_non, mode='L') 


In [ ]:
display(Image_Gaus_x,Image_Gaus_y,Image_Gaus_dx,Image_Gaus_dy,Image_magnitude,Image_supressed)


In [ ]:
# gaussian(np.array([-3,-2,-1,0,1,2,3]),1)
# def gaussian_dx(size, sigma):
#     x = np.linspace(-size // 2, size // 2, size)
#     exponent = -(x**2) / (2 * sigma**2)
#     mask = (-x / (sigma**3 * np.sqrt(2 * np.pi))) * np.exp(exponent)
#     return mask
# def guassion_dy(size, sigma):
#     y = np.linspace(-size // 2, size // 2, size)
#     exponent = -(y**2) / (2 * sigma**2)
#     mask = (-y / (sigma**3 * np.sqrt(2 * np.pi))) * np.exp(exponent)
    
#     return mask
# def gaussian(x, sigma):
#     return -x**2 / ( np.exp(2* (sigma**2)))